In [ ]:
import os.path
from os import path
import numpy as np
import matplotlib.pyplot as pyplot
import gc
import pickle
import random
import copy
import math
from itertools import permutations
import matplotlib.pyplot as plt
from scipy.stats import rv_discrete

Fill in the following paths that contain:

path_neut_amps: array of pulse amplitudes in order to create the neutron spectrum\
path_gamm_amps: array of pulse amplitudes in order to create the gamma spectrum

Provide the following paths in which values will be saved:

path_simulated_neutron_array: Creates neutron pulses used for either training, validation, or testing\
path_simulated_gamma_array: Creates gamma pulses used for either training, validation, or testings\
path_simulated_neutron_template: Creates a template of the average neutron\
path_simulated_gamma_template: Creates a template of the average gamma

In [ ]:
def generate_neutron(pdf,bin_centers,return_params = False, pulse_length = max_length, noise=True, starting_index = 0):
  amp_n = np.random.multivariate_normal(amp_n_mean, amp_n_cov)
  tau_n = np.random.multivariate_normal(tau_n_mean, tau_n_cov)
  t = np.linspace(0,(pulse_length + 300 - 1)*Ts,pulse_length + 300)
  neutron = -amp_n[0]*np.exp(-t/tau_n[0]) + amp_n[1]*np.exp(-t/tau_n[1]) + amp_n[2]*np.exp(-t/tau_n[2])
  neutron = [max(0,x) for x in neutron]
  neutron = neutron/max(neutron)
  total_amp_n = np.random.choice(bin_centers, p=pdf)

  neutron = neutron*total_amp_n
  for i in range(pulse_length):
    if neutron[i] > 0:
      idx = i
      break
  new_neutron = np.zeros(pulse_length)
  new_neutron[starting_index + 1:] = neutron[idx:idx + pulse_length - 1 - starting_index]
  rise_time_samples = np.argmax(new_neutron)
  rise_time_ns_n = rise_time_samples - starting_index
  total_tau_n = (amp_n[1]*tau_n[1] + amp_n[2]*tau_n[2])/(amp_n[1] + amp_n[2])*1e9 # In nano-seconds !
  if noise == True:
    wgn = np.random.normal(noise_mean, noise_std, size=pulse_length)
    new_neutron += wgn

  if return_params == True:
    return new_neutron, total_amp_n, total_tau_n, rise_time_ns_n
  else:
    return new_neutron

def generate_gamma(pdf,bin_centers,return_params = False, pulse_length = max_length, noise=True, starting_index = 0):
  amp_g = np.random.multivariate_normal(amp_g_mean, amp_g_cov)
  tau_g = np.random.multivariate_normal(tau_g_mean, tau_g_cov)
  t = np.linspace(0,(pulse_length + 300 - 1)*Ts,pulse_length + 300)
  gamma = -amp_g[0]*np.exp(-t/tau_g[0]) + amp_g[1]*np.exp(-t/tau_g[1]) + amp_g[2]*np.exp(-t/tau_g[2]) + amp_g[3]*np.exp(-t/tau_g[3])
  gamma = [max(0,x) for x in gamma]
  gamma = gamma/max(gamma)
  total_amp_g = np.random.choice(bin_centers, p=pdf)

  gamma = gamma*total_amp_g
  for i in range(pulse_length):
    if gamma[i] > 0:
      idx = i
      break
  new_gamma = np.zeros(pulse_length)
  new_gamma[starting_index + 1:] = gamma[idx:idx + pulse_length - 1 - starting_index]
  rise_time_samples = np.argmax(new_gamma)
  rise_time_ns_g = rise_time_samples - starting_index
  total_tau_g = (amp_g[1]*tau_g[1] + amp_g[2]*tau_g[2] + amp_g[3]*tau_g[3])/(amp_g[1] + amp_g[2] + amp_g[3])*1e9 # In nano-seconds !
  if noise == True:
    wgn = np.random.normal(noise_mean, noise_std, size=pulse_length)
    new_gamma += wgn

  if return_params == True:
    return new_gamma, total_amp_g, total_tau_g, rise_time_ns_g
  else:
    return new_gamma

Ts = 1e-9 # Sampling period
noise_mean = 0
noise_std = 0.001 # AWGN standard deviation
starting_index = 0
Num_neutrons = 20000 # Number of samples
Num_gammas = 20000 # Number of samples
pulse_lengths = 19000 # Number of samples

tau_n_mean = [17.8e-9, 3193e-9, 570e-9]
tau_n_cov = [[0.0004e-18, 0, 0],
             [0, 100e-18, 0],
             [0, 0, 100e-18]]
amp_n_mean = [1.66, 0.35, 0.62]
amp_n_cov = [[ 0.0256, 0, 0.0029056],
            [ 0,  0.0009,  -0.0004008],
            [ 0.0029056,  -0.0004008,  0.0016]]
total_amp_n_mean = 16.9
total_amp_n_sigma = 1
n_max_length = int(5*max(tau_n_mean)/Ts)

tau_g_mean = [7.35e-9, 918e-9, 10.2e-9, 43e-9]
tau_g_cov = [[0.0025e-18, 0, 0, 0],
             [0, 100e-18, 0, 0],
             [0, 0, 0.0004e-18, 0],
             [0, 0, 0, 0.25e-18]]

amp_g_mean = [15.38, 0.3, 6.69, 0.93]
amp_g_cov = [[ 17.3056, 0, 8.712288, 0],
            [ 0,  0.0009,  -0.0273078, 0],
            [ 8.712288,  -0.0273078,  5.4756, -0.03142152],
            [0, 0, -0.03142152, 0.0324]]
total_amp_g_mean = 20.5
total_amp_g_sigma = 3.5
g_max_length = int(5*max(tau_g_mean)/Ts)
max_length = max([n_max_length,g_max_length])

neut_amps = np.load(path_neut_amps)
gamm_amps = np.load(path_gamm_amps)

hist_data, bin_edges = np.histogram(neut_amps, bins=1000, density=True)
bin_centers_neutrons = 0.5 * (bin_edges[1:] + bin_edges[:-1])
pdf_neutrons = hist_data / np.sum(hist_data)

hist_data, bin_edges = np.histogram(gamm_amps, bins=1000, density=True)
bin_centers_gammas = 0.5 * (bin_edges[1:] + bin_edges[:-1])
pdf_gammas = hist_data / np.sum(hist_data)

simulated_neutron_array = np.zeros((Num_neutrons,pulse_lengths)) # array of simulated neutrons
simulated_gamma_array = np.zeros((Num_gammas,pulse_lengths)) # array of simulated gammas

for i in range(Num_neutrons):
  simulated_neutron_array[i,:] += generate_neutron(pdf_neutrons,bin_centers_neutrons,return_params = False, pulse_length = pulse_lengths, noise=False, starting_index = 0)

for i in range(Num_gammas):
  simulated_gamma_array[i,:] += generate_gamma(pdf_gammas,bin_centers_gammas,return_params = False, pulse_length = pulse_lengths, noise=False, starting_index = 0)

simulated_neutron_template = np.mean(simulated_neutron_array, axis=0)
simulated_neutron_template = simulated_neutron_template/np.max(simulated_neutron_template) # Template of the simulated neutrons

simulated_gamma_template = np.mean(simulated_gamma_array, axis=0)
simulated_gamma_template = simulated_gamma_template/np.max(simulated_gamma_template) # Template of the simulated gammas

np.save(path_simulated_neutron_array,simulated_neutron_array)
np.save(path_simulated_gamma_array,simulated_gamma_array)
np.save(path_simulated_neutron_template,simulated_neutron_template)
np.save(path_simulated_gamma_template,simulated_gamma_template)
